## I/O

Demonstrate several examples of configuring/adding backends for PLINK dataset import.

In [1]:
import sys
import os
sys.path.append(".")
from lib import api
import pandas as pd
import xarray as xr
from pathlib import Path
xr.set_options(display_style='html');

In [2]:
# Path to PLINK dataset for demonstration
path = os.path.join(os.environ['DATA_DIR'], 'gwas/tutorial/1_QC_GWAS/HapMap_3_r3_1')
path

'/home/eczech/data/gwas/tutorial/1_QC_GWAS/HapMap_3_r3_1'

### Choosing Backends

Like pandas and xarray, settings for I/O backends are configured with a global options system (in this case, they are literally using Pandas options).

An advantage of this is that the options themselves are callable python objects with docstrings, which makes interactive exploration easier.  This also provides a convenient way to scope settings globally, locally with a context manager, or on a per-call basis.

In [3]:
# Most backends will default to 'auto' so that they can be chosen based on what's installed
api.describe_option('io.plink.backend')

io.plink.backend The default PLINK reader/writer backend. Available options: 'pysnptools', default is 'auto'
    [default: auto] [currently: auto]


In [4]:
# Set an option globally to use pysnptools for PLINK I/O (https://github.com/MicrosoftGenomics/PySnpTools)
api.set_option('io.plink.backend', 'pysnptools')
api.describe_option('io.plink.backend')

io.plink.backend The default PLINK reader/writer backend. Available options: 'pysnptools', default is 'auto'
    [default: auto] [currently: pysnptools]


In [6]:
# Use global options set above to read a file
ds = api.read_plink(path, chunks='auto', fam_sep=' ', bim_sep='\t')

# OR Define context manager for local scope
with api.option_context('io.plink.backend', 'pysnptools'):
    ds = api.io.read_plink(path, chunks='auto', fam_sep=' ', bim_sep='\t')
    
# OR Set backend for each call
ds = api.io.read_plink(path, chunks='auto', backend='pysnptools', fam_sep=' ', bim_sep='\t')

ds

<xarray.Dataset>
Dimensions:     (sample: 165, variant: 1457897)
Coordinates:
  * variant     (variant) int64 0 1 2 3 4 ... 1457893 1457894 1457895 1457896
  * sample      (sample) int64 0 1 2 3 4 5 6 7 ... 158 159 160 161 162 163 164
Data variables:
    data        (variant, sample) int8 dask.array<chunksize=(813440, 165), meta=np.ndarray>
    is_masked   (variant, sample) bool dask.array<chunksize=(813440, 165), meta=np.ndarray>
    sample_id   (sample) int64 1328 1377 1349 1330 1444 ... 1346 1375 1349 1459
    fam_id      (sample) object 'NA06989' 'NA11891' ... 'NA10854' 'NA12865'
    pat_id      (sample) object '0' '0' '0' '0' ... '0' '0' 'NA11839' 'NA12874'
    mat_id      (sample) object '0' '0' '0' '0' ... '0' '0' 'NA11840' 'NA12875'
    is_female   (sample) int64 2 1 1 2 1 2 1 1 2 1 1 1 ... 1 1 1 2 2 2 1 1 1 2 2
    phenotype   (sample) int64 2 2 1 2 -9 -9 2 -9 1 1 ... 2 1 2 -9 -9 2 1 -9 -9
    contig      (variant) int64 1 1 1 1 1 1 1 1 1 ... 25 25 25 25 25 25 25 25 25
    variant_id  (variant) object 'rs2185539' 'rs11510103' ... 'rs1973881'
    cm_pos      (variant) int64 0 0 0 0 0 0 0 0 0 0 0 ... 0 0 0 0 0 0 0 0 0 0 0
    pos         (variant) int64 556738 557616 718814 ... 154879211 154883690
    a1          (variant) object 'T' 'G' 'T' 'A' 'A' 'C' ... 'T' 'G' 'G' 'A' 'G'
    a2          (variant) object 'C' 'A' 'C' 'G' 'G' 'T' ... 'G' 'T' 'A' 'G' 'A'
Attributes:
    type:     <class 'lib.core.GenotypeCountDataset'>

### Using Custom Backends

Adding unmanaged extension mechanisms for custom backends like those used in Pandas or exposed as a part of the [Astropy I/O Registry](https://docs.astropy.org/en/stable/io/registry.html#io-registry) can be done at runtime.  These are "unmanaged" in the sense that it is entirely up to the user to install dependencies and make them compatible.

In [8]:
# Create a custom backend that just reads PLINK pedigree data

class CustomPLINKBackend(api.io.Backend):
    
    id = 'custom-backend'
    
    def read_plink(self, path):
        cols = names=['sample_id', 'fam_id', 'pat_id', 'mat_id', 'is_female', 'phenotype']
        return pd.read_csv(path + '.fam', sep=' ', names=cols)

In [12]:
# Register the backend and use it
api.io.register_backend(CustomPLINKBackend())

df = api.read_plink(path, backend='custom-backend')
df

,sample_id,fam_id,pat_id,mat_id,is_female,phenotype
0,1328,NA06989,0,0,2,2
1,1377,NA11891,0,0,1,2
2,1349,NA11843,0,0,1,1
3,1330,NA12341,0,0,2,2
4,1444,NA12739,NA12748,NA12749,1,-9
...,...,...,...,...,...,...
160,1447,NA12752,NA12760,NA12761,1,-9
161,1346,NA12043,0,0,1,2
162,1375,NA12264,0,0,1,1
163,1349,NA10854,NA11839,NA11840,2,-9
